In [1]:
# ==================================================================================================================
                                # # # ANALISE DE DADOS FACT ORDER DETAILS # # #
# ==================================================================================================================
# Import de bibliotecas
from extracao_dados import extract_postgres_datawarehouse
import plotly.express as px

# ==================================================================================================================
# Leitura dos dados do data warehouse
fact_order_details = extract_postgres_datawarehouse("fact_order_details")
dim_products = extract_postgres_datawarehouse("dim_products")
dim_suppliers = extract_postgres_datawarehouse("dim_suppliers")
dim_categories = extract_postgres_datawarehouse("dim_categories")

# ==================================================================================================================
# Realizar joins para criar DataFrame consolidado
df = fact_order_details.merge(dim_products, on='product_id')
df = df.merge(dim_suppliers, on='supplier_id')
df = df.merge(dim_categories, on='category_id')

# Criar colunas e medidas calculadas
df['total_sales'] = df['unit_price_x'] * df['quantity'] * (1 - df['discount'])

# ==================================================================================================================
# Criação de visualizações dos dados
# Vendas por Produto
fig_product_sales = px.bar(
    df.groupby('product_name')['total_sales'].sum().reset_index(),
    x='product_name', y='total_sales',
    title='Vendas por Produto'
)
fig_product_sales.show()

# Quantidade Vendida por Categoria
fig_category_quantity = px.bar(
    df.groupby('category_name')['quantity'].sum().reset_index(),
    x='category_name', y='quantity',
    title='Quantidade Vendida por Categoria'
)
fig_category_quantity.show()

# Desconto Médio por Produto
fig_avg_discount = px.bar(
    df.groupby('product_name')['discount'].mean().reset_index(),
    x='product_name', y='discount',
    title='Desconto Médio por Produto'
)
fig_avg_discount.show()

# Mapa de Vendas por Região
fig_sales_map = px.scatter_geo(
    df,
    locations='country',
    locationmode='country names',
    size='total_sales',
    title='Mapa de Vendas por Região'
)
fig_sales_map.show()

# Análise de Desempenho de Fornecedores
fig_supplier_performance = px.pie(
    df.groupby('company_name')['total_sales'].sum().reset_index(),
    names='company_name', values='total_sales',
    title='Desempenho dos Fornecedores'
)
fig_supplier_performance.show()

# ==================================================================================================================
# Relatórios da tabela fact_order_details
# Total de vendas por produtos
df.groupby('product_name')['total_sales'].sum().reset_index()\
    .to_csv('analises/order_details/relatorio_vendas_por_produto.csv', index=False)

# Total de quantidade vendidas por categoria
df.groupby('category_name')['quantity'].sum().reset_index()\
    .to_csv('analises/order_details/relatorio_quantidade_vendida_por_categoria.csv', index=False)

# Total de desconto medio por produto
df.groupby('product_name')['discount'].mean().reset_index()\
    .to_csv('analises/order_details/relatorio_desconto_medio_por_produto.csv', index=False)

# Total de vendas por fornecedor
df.merge(dim_products[['product_id', 'supplier_id']], on='product_id')\
    .groupby('company_name')['total_sales'].sum().reset_index()\
    .to_csv('analises/order_details/relatorio_vendas_por_fornecedor.csv', index=False)

# Total de produtos com menor estoque
df.merge(dim_products[['product_id', 'product_name', 'units_in_stock']], on='product_id')\
    .sort_values(by='units_in_stock_y', ascending=True).head(10)\
    .to_csv('analises/order_details/relatorio_produtos_com_menor_estoque.csv', index=False)

c:\Users\Taking\OneDrive - taking\Área de Trabalho\Projeto Esteira BI\Projeto\hml_config\extracao_dados.py:147: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [2]:
# ==================================================================================================================
                                # # # ANALISE DE DADOS FACT ORDERS # # #
# ==================================================================================================================
# Carregar dados das tabelas
fact_orders = extract_postgres_datawarehouse('fact_orders')  
dim_customers = extract_postgres_datawarehouse('dim_customers')
dim_employees = extract_postgres_datawarehouse('dim_employees')
dim_employee_territories = extract_postgres_datawarehouse('dim_employee_territories')
dim_territories = extract_postgres_datawarehouse('dim_territories')
dim_region = extract_postgres_datawarehouse('dim_region')

# ==================================================================================================================
# Realizar joins para criar DataFrame consolidado
df_2 = fact_orders.merge(dim_customers, on='customer_id', how='left') \
                .merge(dim_employees, on='employee_id', how='left') \
                .merge(dim_employee_territories, on='employee_id', how='left') \
                .merge(dim_territories, on='territory_id', how='left') \
                .merge(dim_region, on='region_id', how='left')

# Exemplo: Calcular total de vendas por região
df_2['total_sales'] = df_2['freight']  # Suponho que 'freight' == 'frete' representa o total de vendas
# ==================================================================================================================
# Criação de visualizações dos dados

# Total de Vendas por Região
sales_by_region = df_2.groupby('region_description')['total_sales'].sum().reset_index()
fig1 = px.bar(sales_by_region, x='region_description', y='total_sales',
              title='Total de Vendas por Região', labels={'region_description': 'Região', 'total_sales': 'Total de Vendas'})
fig1.show()

# Total de Vendas por Funcionário
sales_by_employee = df_2.groupby(['first_name', 'last_name'])['total_sales'].sum().reset_index()
fig2 = px.bar(sales_by_employee, x='first_name', y='total_sales', color='last_name',
              title='Total de Vendas por Funcionário', labels={'first_name': 'Nome do Funcionário', 'total_sales': 'Total de Vendas'})
fig2.show()

# Total de Vendas por Cliente
sales_by_customer = df_2.groupby('company_name')['total_sales'].sum().reset_index()
fig3 = px.bar(sales_by_customer, x='company_name', y='total_sales',
              title='Total de Vendas por Cliente', labels={'company_name': 'Cliente', 'total_sales': 'Total de Vendas'})
fig3.show()


# Total de Vendas por Território
sales_by_territory = df_2.groupby('territory_description')['total_sales'].sum().reset_index()
fig4 = px.bar(sales_by_territory, x='territory_description', y='total_sales',
              title='Total de Vendas por Território', labels={'territory_description': 'Território', 'total_sales': 'Total de Vendas'})
fig4.show()


# Total de Vendas por Data de Pedido
sales_by_order_date = df_2.groupby('order_date')['total_sales'].sum().reset_index()
fig5 = px.line(sales_by_order_date, x='order_date', y='total_sales',
               title='Total de Vendas por Data de Pedido', labels={'order_date': 'Data do Pedido', 'total_sales': 'Total de Vendas'})
fig5.show()

# ==================================================================================================================
# Relatorios fact_orders
# Total de Vendas por Região
sales_by_region.to_csv('analises/orders/relatorio_vendas_por_regiao.csv', index=False)
# Total de Vendas por Funcionário
sales_by_employee.to_csv('analises/orders/relatorio_vendas_por_funcionario.csv', index=False)
# Total de Vendas por Cliente
sales_by_customer.to_csv('analises/orders/relatorio_vendas_por_cliente.csv', index=False)
# Total de Vendas por Território
sales_by_territory.to_csv('analises/orders/relatorio_vendas_por_territorio.csv', index=False)
# Total de Vendas por Data de Pedido
sales_by_order_date.to_csv('analises/orders/relatorio_vendas_por_data_de_pedido.csv', index=False)



c:\Users\Taking\OneDrive - taking\Área de Trabalho\Projeto Esteira BI\Projeto\hml_config\extracao_dados.py:147: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

